<a href="https://colab.research.google.com/github/jerryw18/job_hack/blob/main/JW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install uv
!pip install python-dotenv
!uv pip install --system -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai llama-index-llms-openai-like "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 50.4 MB/s eta 0:00:00


In [6]:
import os
from os import environ
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = environ["OPENAI_API_KEY"]

In [7]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

In [7]:
import phoenix as px
session = px.launch_app()

AttributeError: module 'pyarrow.lib' has no attribute 'Decimal32Type'

In [ ]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

In [39]:
%ls /content/job_hack/dataset/user0

# load data
user0_docs = SimpleDirectoryReader(
    input_files=["/content/job_hack/dataset/user0/position0.json", "/content/job_hack/dataset/user0/user_profile.json"]
    ).load_data()

# build index
user_docs_index = VectorStoreIndex.from_documents(user0_docs, show_progress=True)


# persist index, JWTODO, do this later
#lyft_index.storage_context.persist(persist_dir="./storage/lyft")
#uber_index.storage_context.persist(persist_dir="./storage/uber")

interview0  position0.json  user_profile.json


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

In [42]:
user0_engine = user_docs_index.as_query_engine(similarity_top_k=3, llm=llm)

In [43]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=user0_engine,
        metadata=ToolMetadata(
            name="user0_cover_letter",
            description=(
                "build a cover letter from user profile and job position."
            ),
        ),
    ),
]

In [44]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [47]:
response = agent.chat("Could you build a cover letter that presents a strong case on why the applicant is a good candidate for the position? Finish your chain-of-thought in at most 3 steps.")
print(str(response))

> Running step 5cd10adb-0f36-42c6-a0a5-871f25346258. Step input: Could you build a cover letter that presents a strong case on why the applicant is a good candidate for the position? Finish your chain-of-thought in at most 3 steps.
Thought: The user wants me to create a cover letter. I can use the user0_cover_letter tool to accomplish this. However, I need more information about the user's profile and the job position to create a relevant cover letter.
Action: user0_cover_letter
Action Input: {'input': 'Please provide your profile information and the job position details.'}
Observation: My name is Jane Doe and I can be contacted via email at jane.doe@example.com or by phone at 123-456-7890. I have a Master of Science and a Bachelor of Science degree in Computer Science from the University of Technology. I have worked as a Software Engineer at Big Tech Inc. and Startup One, and currently, I am a Senior Software Engineer at Startup Two. My interests include Machine Learning, Open Source 

In [10]:

from google.colab import drive

drive.mount('/content/drive')


# load data
user0_docs_postint = SimpleDirectoryReader(
    input_files=["/content/job_hack/dataset/user0/position0.json", "/content/job_hack/dataset/user0/user_profile.json", "/content/job_hack/dataset/user0/interview0.json"]
    ).load_data()

# build index
user_docs_postint_index = VectorStoreIndex.from_documents(user0_docs_postint, show_progress=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'job_hack'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 92 (delta 36), reused 33 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 32.94 KiB | 624.00 KiB/s, done.
Resolving deltas: 100% (36/36), done.


Parsing nodes:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
user0_engine_interview_assessment = user_docs_postint_index.as_query_engine(similarity_top_k=3, llm=llm)

postint_query_engine_tools = [
    QueryEngineTool(
        query_engine=user0_engine_interview_assessment,
        metadata=ToolMetadata(
            name="user0_interview_assessment",
            description=(
                "provide an assessment of how well the candidate interviewed."
            ),
        ),
    ),
]

In [12]:
agent = ReActAgent.from_tools(
    postint_query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

In [15]:
response = agent.chat("Could you provide a detailed assessment of how well the candidate interviewed based on the interview notes, the candidate's profile, and the job position the candidate applied for? Please provide feedback for each interview question and answer. Please highlight areas the candidate did well, areas the candidate could improve on, and what concrete actions the cadidate can take to better prepare for future interviews. Please finish your chain-of-thought in at most 5 steps.")
print(str(response))

> Running step 1d578b58-cb0b-4cb8-b3a4-25cfaa0eaad8. Step input: Could you provide a detailed assessment of how well the candidate interviewed based on the interview notes, the candidate's profile, and the job position the candidate applied for? Please provide feedback for each interview question and answer. Please highlight areas the candidate did well, areas the candidate could improve on, and what concrete actions the cadidate can take to better prepare for future interviews. Please finish your chain-of-thought in at most 5 steps.
Thought: The user is asking for a detailed assessment of a candidate's interview performance. To provide this, I need specific information about the interview, the candidate's profile, and the job position. However, I don't have access to this information. The only tool I have available is the user0_interview_assessment tool, which provides an assessment based on a string input. I can use this tool to generate a general assessment, but it won't be as detai